In [ ]:
# load standard packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import re
import missingno as msno

In [ ]:
plt.style.use('pgdatastyle.mplstyle') 

In [ ]:
cast_df = pd.read_csv('cast.csv', low_memory = False, delimiter = ',')

In [ ]:
cast_df.columns

In [ ]:
cast_df.head()

#### The above file contains metadata about a given cast (cast station location in long/lat coordinates, date/time of cast, ship info, general ambient weather conditions). The cast ID (Cst_Cnt) is referenced in the bottle data and provides a way of linking this metadata to the physical measurements in the bottle data.

In [ ]:
bottle_df = pd.read_csv('bottle.csv', low_memory= False)

In [ ]:
bottle_df.columns

#### This is pretty nice. There's a lot of data here with measurements of various quantities. Information on some of these quantities is listed below:

## Data Definitions

| Measurement | Explanation | 
| --- | --- | 
| Salinity (Salnty) | Measured in grams of salt per 1000 mL seawater. Often reported in parts per thousand.  |
| Water Temperature (T_degC) | The local temperature (at a specific depth) of the seawater in Celsius |
| Oxygen Level (O2ml_L) | The concentration of dissolved oxygen (ml/L) in seawater. 
| Phosphate Level (PO4uM) | Concentration of dissolved PO4 ion in seawater (units: micromol/L) |
| Silicate Level (SiO3uM) | Concentration of silicate ion SiO3 in seawater (units: micromol/L) |
| Nitrogen Dioxide Level (NO2uM) | Concentration of nitrogen dioxide in seawater (units: micromol/L) |
| Ammonia Level (NH3uM) | Concentration of ammonia in seawater (units: micromol/L) | 
| Nitrate Level (NO3uM) | Concentration of nitrate ion NO3 in seawater (units: micromol/L) |
| Chlorophyll A (ChlorA) | Concentration of Chlorophyll A in seawater (units: microgram/L) |
| Phaeopigments(Phaeop) | Concentration of phaeopigments in seawater. (units: microgram/L) |
| Sea level depth (Depthm) | Depth at which bottle measurement was taken (units: meter) |

Other relevant definitions are:

| Measurement | Explanation | 
| --- | --- |
| Cast (Cst_Cnt) | Cast ID number. |
| Btl_Cnt | Bottle identifier (for a given cast) for measurements taken as a function of depth.  |

Other relevant columns will be the lat/longitude of the cast, and the datetime.

## Massaging the metadata
Let's inspect the cast data a little more closely and then clean it up and subset it.

In [ ]:
print(cast_df.shape)

In [ ]:
cast_df.info()

In [ ]:
print(cast_df.describe().T)

Alright, so we can see that the most important metadata (lat/long and datetimes) have no NaNs. There are very few NaNs in the Wind Direction / Wind Speed data. So we might want to keep these last two measurements. We'll also want the Cast Counts and Station IDs as these appear in both the Cast and Bottle measurement dataframes.

In [ ]:
cast_df_subcols = ['Cst_Cnt', 'Sta_ID', 'Date', 'Time', 'Lat_Dec', 'Lon_Dec', 'Wind_Dir', 'Wind_Spd']
cast_df = cast_df[cast_df_subcols]
print(cast_df.head(10))

#### Some obvious problems are the Time column (the generic date out front that doesnt make sense). There are also some NaNs in the time column that need to be dealth with. Also Date's and Time dtype is an object. We want to combine these columns into a single datetime

In [ ]:
cast_df['Time'] = cast_df.loc[:,'Time'].str.replace('12/30/1899 ', '')
print(cast_df['Time'].head())

In [ ]:
cast_df['Time'].isna().sum()

We had better deal with these NaNs. In general, hydrographic sampling doesn't depend too much on the time of day (otherwise, one might expect pretty frequent sampling at each station). While we kept the time of day (why throw it out?), we don't want to throw out data for which we don't have the time that the data was taken. For these we will just fill in with noon or '12:00:00'.

In [ ]:
cast_df['Time'].fillna('12:00:00', inplace = True)
cast_df['Time'].isna().sum()

In [ ]:
cast_df['DateTime'] = pd.to_datetime(cast_df['Date'] + ' ' + cast_df['Time'])
cast_df = cast_df.drop(columns = ['Date', 'Time'])
print(cast_df.head(10))

In [ ]:
print(cast_df['DateTime'].dtype)

In [ ]:
cast_df.isna().sum()

Some NaNs in the Wind Spd and Wind Direction. Let's visualize this to see if we can/should do a simple impute on this data.

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].hist(cast_df['Wind_Spd'], bins = 15)
ax[0].set_ylabel('Wnd_Spd')
ax[1].hist(cast_df['Wind_Dir'], bins = 15)
ax[1].set_ylabel('Wnd_Dir')
plt.show()

#### The median is probably a pretty OK place to impute on the Wind speed. But The Wind_Dir is strongly peaked 32-34 degrees.

In [ ]:
print(cast_df['Wind_Spd'].median())
cast_df['Wind_Spd'].fillna(cast_df['Wind_Spd'].median(), inplace = True)

In [ ]:
m = cast_df['Wind_Dir'].mode().values[0]
cast_df['Wind_Dir'].fillna(m, inplace = True)

In [ ]:
cast_df[['Wind_Spd','Wind_Dir']].isna().sum()

In [ ]:
cast_df.isna().sum()

So far so good. Let's use plotly to visualize the spatial distribution of the measurement stations where the cast were made. 

In [ ]:
fig = px.scatter_geo(cast_df, lat = 'Lat_Dec', lon = 'Lon_Dec')
fig.update_geos(fitbounds = 'locations', resolution=50, showcoastlines=True, coastlinecolor="Red", showland=True, landcolor="ForestGreen", showocean=True, oceancolor="MidnightBlue")
fig.update_traces(marker = dict(size = 2, color = 'yellow'))
fig.update_layout(title = 'CalCoFI Monitoring Locations', height=300, width = 500, margin={"r":10,"t":30,"l":10,"b":30})


Now let's deal with the bottle_df now. We are going to subset the bottle data.

In [ ]:
# This is a large number of columns
bottle_df.columns

In [ ]:
bottle_df.shape

In [ ]:
rawmatcher = re.compile('^R_')
rawcols = [colnames for colnames in bottle_df.columns if rawmatcher.search(colnames)]
bottle_df = bottle_df.drop(columns = rawcols)
print(bottle_df.columns)

The 'R_quantity' columns are raw data columns that can be removed (i.e. CoCalFI has done some basic preprocessing for us already).The '_qual' columns or columns like 'NO2q' correspond to a data quality assessment. 0 or NaN on quality columns corresponds to good data. The scale goes to 9 where 9 are essentially sensor malfunction or omission.   

In [ ]:
bottle_df.isna().sum()

OK. It's pretty obvious that there are a whole bunch of NaN dominated columns. Let's drop data that has more than 740K NaNs. These are NaN dominated columns and reflect measurements that have just recently started being taken by the CoCalFI survey, new labeling systems, etc.

In [ ]:
nan_threshold = len(bottle_df) - 740000
bottle_df = bottle_df.dropna(axis = 1, thresh = nan_threshold)

In [ ]:
bottle_df.columns

#### All right, so we still have a bunch of columns left. Of the remaining, many of the columns are data quality/precision assessors which are useful for processing the raw data that we've already dropped. So let's get rid of these and any derived quantities like 'Oxy_umol/Kg', 'O2sat' (derived from O2ml_L). Subsetting on the columns to leave in we have:

In [ ]:
cols_to_keep = ['Cst_Cnt', 'Btl_Cnt', 'Sta_ID', 'Depthm', 'T_degC','Salnty', 'O2ml_L', 'STheta','ChlorA', 'Phaeop', 'PO4uM', 'SiO3uM', 'NO2uM', 'NO3uM']
bottle_df = bottle_df[cols_to_keep]


In [ ]:
merged_df = cast_df.merge(bottle_df, on = ['Cst_Cnt', 'Sta_ID'], how = 'inner')

Let's visualize where the NaNs are in our dataframe.

In [ ]:
msno.matrix(merged_df, sparkline=False, figsize = (10,5), color=(0.27, 0.52, 1.0))

Most of the mineral measurements (nitrates, silicates, phosphates) and photopigment concentrations are NaNs in the first part of the dataset. As the Cst_Cnt is time ordered, the logical conclusion is that most of these measurements were made available after a specific date (i.e. due to technological advances or because they started to become metrics of interest to the community).

Let's drop everything before the year where Chlorophyll/Phaepigment data is available and where the bulk of the mineral data is also available. This is a lot of data to drop, I know...but at least for now we will restrict the study to data where we can justifiably impute. 

In [ ]:
startindex = merged_df[['ChlorA','Phaeop']].first_valid_index()
print(startindex)

In [ ]:
merged_df_restricted = merged_df.loc[startindex::,:]

In [ ]:
# lets visualize missing no now
msno.matrix(merged_df_restricted)

## EDA/Visualization for Imputation

#### A lot of this data is likely correlated. Let's subset the part of the data that we immediately expect to be features (physical bottle measurements as opposed to index data / metadata).

In [ ]:
feat_names = merged_df_restricted.columns.drop(['Cst_Cnt', 'Sta_ID', 'Lat_Dec', 'Lon_Dec', 'Wind_Dir', 'Wind_Spd',
       'DateTime', 'Btl_Cnt'])

In [ ]:

sns.heatmap(merged_df_restricted[feat_names].corr())

In [ ]:
print(merged_df_restricted[feat_names].corr())

### Correlations 
#### Dissolved Oxygen
The dissolved oxygen level (O2ml_L) is strongly anticorrelated with the phosphate (PO4), silicate (SiO3), and nitrate levels (NO3). These minerals -- particularly phosphates -- are often directly related to surface phytoplankton population growth. The strong negative correlation is rather interesting and merits futher investigation. 

The Pearson correlation coefficient also is pretty large (positive) between O2 and Temperature, large (negative) between O2 and Salinity/Depth/STheta (Stheta is the sigma theta which is the seawater density).

#### Temperature
The temperature has a strong negative correlation with the depth (the deeper you go the colder it gets...perhaps not too surprising). 

#### Salinity
The Pearson correlation coefficient between depth and Salinity is positive (possibly indicating the deeper you go the more saline the water). This also seems to be aligned with the correlation between sigma-theta vs depth and sigma-theta vs. salinity.

Almost all these variables are moderately to highly correlated with each other. However, coastal oceans are complex and these relations are likely not so simple. A quick visualization of the relationship of these variables across the entire dataset (as a first go) will be useful (not just for EDA but for possible imputation strategies).

In [ ]:
sns.pairplot(merged_df_restricted[feat_names].sample(10000)) #we downsample or else this would take forever

There is a lot going on here. But here are my key take-aways:
1. The phosphate and nitrate vs. dissolved O2 relationship is well captured by a linear relationship with a negative Pearson correlation coefficient. The silicate vs dissolved O2 largely seems to be as well, but there is a branch of the data that has a positive relationship. Probably worth thinking about why that might be.

2. The phosphate and nitrate concentrations follow a pretty direct linear relationship. On the other hand, the phosphate and nitrate concentrations exhibit an increase for a large range of silicate concentraton and then maximum/decrease for high silicate concentrations. Very clearly, this is a nonlinear relationship (approximal linear in the low-mid silicate range). In fact, the bend in the data looks like its whats responsible for the second positively correlated branch in the SiO3 vs O2 data.


3. The NO3, PO4,SiO3 vs Temp scatterplots clearly show why the Pearson correlation coefficients were highly negative. However, a preliminary look at these scatterplots on the downsampled data show that the relationship is more complex than in the minerals vs. dissolved O2 case.

There is an initial increase in the NO3, PO4,SiO3 vs Temp scatterplots for low temperatures. For a large subset of the data theres a decreasing rolloff to zero. Some subset of the data follows a slower linear decrease to 0.

4. NO2 doesnt really look that interesting. We should drop this column.

5. Chlorophyll and phaeopigments are concentrated within the first 200 m of depth or so. That would make a lot of sense as this is the zone where light can actually reach.

6. The salinity vs other variables (Temp, Stheta, O2, PO4, NO3, SiO3, depth ) shows rather clearly that there are two different subsets of the data following linear trends of opposing slopes. These might correspond directly to the two components of the Gaussian mixture distribution in the salinity histogram (something thats easy to check).

7. The various variable vs depth is quite interesting:
- O2 vs depth first has a decrease then a plumetting down to very low oxygen levels at intermediate depths and then a partial recovery at large depths.
- Temperature vs depth shows a strong decline in intermediate depth regions and then a leveling out at high depths close to 0 C. There is also indcaions of temperature leveling out or slowing down its decline at intermediate depths -- there are clearly multiple types of behavior in this graph.
- PO4 and NO3 rise to maxima at roughly the same depth as O2 gets its minimum then starts slowly decreasing as depth increases (mirroring the slow increase in O2 at large depths). 
- At low depths the salinity start at a higher level or a lower salinity level and then basically asymptote to the same value at higher depths.

#### For O2, PO4, NO3 there is probably three different regions (region of higher O2 at surface depth, then low O2 levels in intermediate depths, and then the O2 recovery region).

Now let's take a closer at the distribution for the various variables on the entire dataset:

In [ ]:
plt.style.use('pgdatastyle.mplstyle')  
for column in feat_names:        
    plt.hist(merged_df_restricted[column], bins = 60, density=True)
    plt.title('Distribution of ' + column)
    plt.grid()
    plt.xlabel(column)
    plt.ylabel('Normalized Count')

    plt.show()

There's a lot going on here. It's pretty clear that the nitrate, silicate, phosphate, temperature, and oxygen distributions are indeed some kind of mixture of distributions. The matching structure of these mixture sub-distributions reinforce the high Pearson correlations between oxygen level and nitrates, silicates, and phosphates and provide stronger evidence of the linkage of these quantities. Also, the fact this looks like roughly three components to the mixture of these distributions, and our previous observations on depth dependence may lend credence to the hypothesis that there is roughly speaking at least a three-layer depth stratification of these quantities.

The salinity looks like a bimodal distribution while the temperature is pretty difficult to describe in simple terms. 

Let's take a closer look at the chloropyll and phaeopigment distributions: 

 

In [ ]:
plt.hist(merged_df_restricted['ChlorA'], range = (0,1), bins = 200)
plt.show()

In [ ]:
sns.boxplot(y='ChlorA', data = merged_df_restricted)

In [ ]:
plt.hist(merged_df_restricted['Phaeop'], range = (0,1), bins = 200)
plt.show()

In [ ]:
sns.boxplot(y='Phaeop', data = merged_df_restricted)

Both Chlorophyll and Phaeopigment concentrations look like there is a component that is Gamma-distributed. Chlorophyll has an additional component at low Chlorophyll values. There are portions of both distributions that are quite extreme and well outside the 1-3 IQR. What is going on with these values? They are likely not measurement errors but values where surface concentrations do indeed become quite extreme. There are a lot more NaNs in these columns. Is it possible that its because these measurements were usually only taken at surface levels (where they are likely non-zero)?

In [ ]:
df1 = merged_df_restricted[['Depthm', 'ChlorA', 'Phaeop']]

In [ ]:
df1[df1['ChlorA'].isna() == True].hist(column = 'Depthm', bins = 60)
plt.xlim(0,1000)

In [ ]:
df1[df1['ChlorA'].notna() == True].hist(column = 'Depthm', bins = 50)
plt.xlim(0,1000)

In [ ]:
sns.scatterplot(x = 'Depthm', y = 'ChlorA', data = df1)

The moral of the above visualizations is that a lot of NaNs in ChlorA correspond to depths > 150 m while most of the recorded data was made at depth <= 200 m. But from the scatterplot most of the recorded Chlorophyll data is 0 anyway. So we'll impute NaNs at those lower depths to 0.

Imputing NaNs within the <= 100 m depth range might be possible depending on the data. There is a high degree of variability in this data and no obvious strong correlations on other features in aggregate. So casts where all the chlorophyll data or most of the surface chlorophyll data is missing probably shouldnt be imputed. We'll just leave those datapoint as NaNs. But there is likely a well defined depth dependence in a single cast. It might be possible to interpolate or extrapolate based off other Chlorophyll data in the cast if these exist and our NaNs are single holes in the cast data record. 

In [ ]:
df1[df1['Phaeop'].isna() == True].hist(column = 'Depthm', bins = 60)
plt.xlim(0,1000)

In [ ]:
df1[df1['Phaeop'].notna() == True].hist(column = 'Depthm', bins = 50)
plt.xlim(0,1000)

In [ ]:
sns.scatterplot(x = 'Depthm', y = 'Phaeop', data = df1)

Pretty much the same story as with chlorophyll. We'll take care of cleaning the chlorophyll snd phaeopigment column in a little bit.

In [ ]:
merged_df_restricted.info()

In [ ]:
merged_df_restricted.loc[ ((merged_df_restricted['ChlorA'].isna() == True) | (merged_df_restricted['Phaeop'].isna() == True) ) & (merged_df_restricted['Depthm'] >=150), ['ChlorA', 'Phaeop']] = merged_df_restricted.loc[ ((merged_df_restricted['ChlorA'].isna() == True) | (merged_df_restricted['Phaeop'].isna() == True) ) & (merged_df_restricted['Depthm'] >=150), ['ChlorA', 'Phaeop']].replace({np.nan: 0})

In [ ]:
merged_df_restricted.info()

In [ ]:
merged_df_restricted = merged_df_restricted.drop(columns = ['NO2uM']) # nothing super interesting is happening with NO2 so im dropping it

In [ ]:
msno.matrix(merged_df_restricted)

In [ ]:
merged_df_restricted[merged_df_restricted['Phaeop'].isna() == True].hist(column = 'Depthm', bins = 5)
plt.xlim(0,1000)

In [ ]:
merged_df_restricted[merged_df_restricted['ChlorA'].isna() == True].hist(column = 'Depthm', bins = 5)
plt.xlim(0,1000)

That looked like it worked.

We've established strong correlations and nearly linear relationships between many of the other features in the dataset. We can take advantage of this for imputation purposes. We'll subset on these features and perform MICE using sklearn's iterative imputer.

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

We dropped silicates for imputation as its dependence on the other variables is not linear. We could do a feature transformation and then use MICE on the transformed set and transform back, but a simpler approach might be just as good for imputation purposes. 

In [ ]:
colsubimp = ['O2ml_L', 'PO4uM', 'NO3uM'] 
data_imp_subset = merged_df_restricted[colsubimp]

In [ ]:
data_imp_subset.head()

In [ ]:
imputer = IterativeImputer(sample_posterior = True)

In [ ]:
imputeddata = imputer.fit_transform(data_imp_subset)

We are leaving the imputed oxygen levels out as we tried this and it skewed the oxygen depth distribution.

In [ ]:
imputeddf = pd.DataFrame(imputeddata, columns=['O2ml_L','PO4uM', 'NO3uM'], index = data_imp_subset.index)
imputeddf.head()

In [ ]:
merged_df_restricted[colsubimp].head()

In [ ]:
merged_df_restricted[['PO4uM', 'NO3uM']] = imputeddf[['PO4uM', 'NO3uM']] #not putting in O2 imputed values


In [ ]:
merged_df_restricted.head()

In [ ]:
merged_df_restricted.info()

Still some NaNs in ChlorA, Phaeop, SiO3uM and Salnty. Before we deal with these, let's save our current dataframe to file. The iterative imputer with sampling the posterior took a little while and we've done a good amount of data transformations. Would be good to start from a saved csv file for the rest of the cleaning.


In [ ]:
merged_df_restricted.to_csv('intermediate_df.csv')